# Geofences mòbils

- [ ] deuria solucionar uniformitat noms columnes upper/lower

- [X] Crear punts geometria al voltant embarcació
- [X] Crea geofence actualitze per a cada posició
- [X] Representa visualment la geofence mòbil
- [X] Detecta entrades en geofence mòbil per a MMSI practicatge
- [ ] Generalitza bucle per a servei practicatge / embarcacions usuaries
- [ ] Geometria acord a l'embarcació

In [ ]:
import pandas as pd     # Biblioteca tractament de dades en taules
import numpy  as np     # Biblioteca arrays i funcions matemàtiques
import pprint as pp     # Biblioteca imprimir en format més llegible algunes dades
import folium           # Biblioteca creació mapes interactius
import picket           # Cal fitxer picket.py en pwd ja que defineix classe Fence (geofencing)
import json             # jsonObjecte = json.dumps(FitxaDic, indent = 6)  # print(jsonObjecte)
from Moduls import geofences as geo # Funcions pròpies: generals, geofences, escales, plànols, #kml?
from Moduls import constants as cte # Constants pròpies: geofences, MMSIs pràctics

## Funcions generals
- Guardar mapes

In [ ]:
def DataInstant():
    date = !date +%F_%R
    return (str(date).strip("[]'"))
DataInstant()

## Funció per guardar els mapes en html. 
## Li passem objecte i nom que volem. Guardarà amb data "nom_%y-%m-%d_%H:%M.html"
def GuardaHTML(objMapa,nom):
    objMapa.save(outfile = str('./Resultats/' + nom + "_" + DataInstant() + '.html') )

## Càrrega dades, importació de fitxer csv i json. 
- Des de fitxa

In [ ]:
!pwd
!cd Resultats/; ls -hs | grep Fitxa # -s - show size   # --sort=size (or -S) - sort by file size.  # --sort=time ( or -t) - sort by modification time.

elecc = input("Quina fitxa vols gastar?")
try:
    fit = open("./Resultats/"+elecc.split()[1].strip(),"r")
except IndexError:
    fit = open("./Resultats/"+elecc.strip(),"r")    
Fitxa = json.load(fit)
fit.close()

- Des de taula

In [ ]:
dfUDP_4  = pd.read_csv("./Resultats/2021-10-22_19:41_DataFrame_AIS_date_separat_4dies.csv",delimiter=";") # 2021-10-22_19:41_DataFrame_AIS_date_separat_4dies.csv

In [ ]:
# Trajectoria_pilot_df["DATE_DT"] = Trajectoria_pilot_df["DATETIME"].astype('datetime64[ns]')
# dfUDP["Date_str"] = dfUDP["Date_dt"].astype(str)
# Trajectoria_cargo_dic["DATE_DT"] = pd.to_datetime(Trajectoria_cargo_dic["DATE_STR"], format = "%Y/%m/%d %H:%M:%S", errors = "coerce")

In [ ]:
dfUDP_25 = pd.read_csv("./Resultats/Dataframe_Basic_25dies.csv",
                       index_col="Unnamed: 0",     # Columna index creada sense nom
                       parse_dates = ["DATETIME"], # Inclou columna dates com timestamp per poder operar amb timedeltes
                       na_values = ["-"])          # Si convertim els "-" en NaN, la columna sencera serà float i no str
                                                   # Igualment cal fer .dropna() abans de passar-ho a folium o no va
# Dataframe_Basic_25dies.csv
# 2021-11-19_13:09_DataFrame_AIS_datetime_25dies.csv

- Trobem totes les embarcacions practicatge (tipus 50,51,52) als distints fitxers

In [ ]:
# ## Busquem als fitxers convertits en DataFrames
tipo_buque = [50,51,52]
for tipo in tipo_buque:
    unics1 = list(dfUDP_4[   dfUDP_4["TipoBuque"] == tipo]["name"].unique())
    unics2 = list(dfUDP_25[dfUDP_25["TIPOBUQUE"] == tipo]["NAME"].unique())
    print("\nTipo",tipo,end=" : ")    
    for nom in set(unics1+unics2):
        print(nom.strip(),end=", ")

In [ ]:
## Busquem als fitxers convertits en Diccionaris
unics = []
for m in Fitxa.keys():
    a = set(Fitxa[str(m)]["TIPOBUQUE"]).intersection(set(tipo_buque))
    if len(a)>0:
        unics.append((Fitxa[str(m)]["NAME"],Fitxa[str(m)]["TIPOBUQUE"]))
sorted(unics, key = lambda tipo: tipo[1])

- Carrega els MMSI de practicatge i MMSI usuari.

In [ ]:
## Guardats en mòdul
MMSIs_tugs = [cte.mmsi_tug_1,cte.mmsi_tug_2,cte.mmsi_tug_3,cte.mmsi_tug_4,cte.mmsi_tug_5,cte.mmsi_tug_6,
              cte.mmsi_tug_7,cte.mmsi_tug_8,cte.mmsi_tug_9,cte.mmsi_tug_10,cte.mmsi_tug_11]
MMSI_pilots = [cte.mmsi_pilot_1, cte.mmsi_pilot_2, cte.mmsi_pilot_3]
# es podria afegir:  MMSIs_SAR si volguerem detectar operació salvament

## Embarcació a qui se li presta el servei
MMSI_usuari = 636017878 # # 477764600 'Cosco pride'
MMSI_servei = cte.mmsi_tug_2

### Generem planols
- Per crear objecte mapa folium amb les tanques incloses

In [ ]:
geo.representaTanques(geo.regeneraMapa(cte.vlc0[0], zoom = 12),cte.tanques)

### Exemples de trajectories per a servei portuari i usuari

In [ ]:
## Podem treballar amb DF o Fitxa:
Trajectoria_usuari_orig_df   = geo.trajectoria(dfUDP_25, MMSI_usuari)
Trajectoria_servei_orig_df   = geo.trajectoria(dfUDP_25, MMSI_servei)
Trajectoria_usuari_orig_dic  = geo.trajectoria(Fitxa,    MMSI_usuari)
Trajectoria_servei_orig_dic  = geo.trajectoria(Fitxa,    MMSI_servei)

- Filtrem per a quedar-nos únicament amb les entrades en que les dos embarcacions tenen emissió.

### Filtre per a dates coincidents (no ens val de res que les coordenades coincidisquen però en moments distints).
- [X] Representar en planol les coordenades per a intervals de temps que tinguen en comú.
- [ ] Desprès representar amb bucle les polilinies amb marcador de la seua data d'emissió.

In [ ]:
def retallarTemps(df_servei_orig, df_usuari_orig):
    if (len(df_servei_orig)>0 and len(df_usuari_orig)>0):
        try:
            Ti_servei = pd.to_datetime(df_servei_orig.iloc[0]["DATETIME"], format = "%Y/%m/%d %H:%M:%S", errors = "coerce")
            Ti_usuari = pd.to_datetime(df_usuari_orig.iloc[0]["DATETIME"], format = "%Y/%m/%d %H:%M:%S", errors = "coerce")
            Tf_servei = pd.to_datetime(df_servei_orig.iloc[len(df_servei_orig)-1]["DATETIME"], format = "%Y/%m/%d %H:%M:%S", errors = "coerce")
            Tf_usuari = pd.to_datetime(df_usuari_orig.iloc[len(df_usuari_orig)-1]["DATETIME"], format = "%Y/%m/%d %H:%M:%S", errors = "coerce")
            prim = max(Ti_servei,Ti_usuari)
            ulti = min(Tf_servei,Tf_usuari)
            df_servei = df_servei_orig[(df_servei_orig["DATETIME"]>prim) & (ulti>df_servei_orig["DATETIME"])]
            df_usuari = df_usuari_orig[(df_usuari_orig["DATETIME"]>prim) & (ulti>df_usuari_orig["DATETIME"])]
            try:
                print("      Files servei / usuari:")
                print("Original:  ",len(Trajectoria_servei_orig_df)," / ",len(Trajectoria_usuari_orig_df),   # Millorar format string
                  "\nFiltrades:  ",len(df_servei)," / ",len(df_usuari),
                  "\nreducció", len(Trajectoria_servei_orig_df)/len(df_servei), " / ", len(Trajectoria_usuari_orig_df)/len(df_usuari))
                return(df_servei,df_usuari)
            except ZeroDivisionError:
                print("Alerta, un dels dataframes té longitud zero. Açò es pot deure a que eixe MMSI no es troba al fitxer o que no hi ha cap temps coincident entre usuari i servei")
                print(df_servei_orig,"\n",df_usuari_orig)
                return(0,0)
        except [IndexError,AttributeError]:
            print("Un dels MMSI dona error, possiblement no es troba en este arxiu.")
            print(df_servei_orig,"\n",df_usuari_orig)
            return(0,0)
    else:
        return(0,0)

Trajectoria_servei_df, Trajectoria_usuari_df = retallarTemps(Trajectoria_servei_orig_df,Trajectoria_usuari_orig_df)

In [ ]:
Traj_usu = geo.trajectoria(dfUDP_25, MMSI_usuari)
display(Traj_usu)
# for mmsi_servei in MMSI_pilots: #MMSIs_tugs: #MMSI_pilots:
Traj_ser = geo.trajectoria(dfUDP_25, 224071170)
print(mmsi_servei)
display(Traj_ser)
dfUDP_25[dfUDP_25["MMSI"]==224071170]
#     Traj_ser, Traj_usu = retallarTemps(Traj_ser,Traj_usu)

In [ ]:
# Trajectoria_usuari_df
# Trajectoria_servei_df

#### Veiem si les trajectories coincideixen en algun moment en el temps (no necessàriament simultàneament)

In [ ]:
mapa = geo.representaTanques(geo.regeneraMapa(cte.vlc0[0]),cte.tanques)

In [ ]:
## Si li passem coordenades que contenen "-" no representa la polylinia en absolut
# colors = ("#789dca","#ff6961","#f3aacb","#77dd77","#fdfd96","#ffc870","#ffe000","#caf000")

In [ ]:
geo.representaTrajectories(mapa,[Trajectoria_usuari_df,Trajectoria_servei_df])

In [ ]:
# Superposem la traectoria original i la filtrada
geo.representaTrajectories(mapa,[Trajectoria_usuari_orig_df,Trajectoria_servei_orig_df],["gray","#FF7B00"])


## Geometria geofence mòbil
- Definim geometria geofence mòbil

In [ ]:
## Rectangle (uns 100 m per costat) centrat en posició instantànea de l'embarcació (punts trajectoria).
#         #### Geofence mòbil #####
#      ______________________________                        -----🚢
#     |              ^              |                       /       
#     |     (X)      | +dy          |                    /         
#     |              |              |                  /            
#     |  -dx         |          +dx |                /              
#   /-|<------------[punt*]-------->|               /                
#  /  |              |              |----\         /                 *(lon_orig,lat_orig)
# /   |              | -dy          |     \       /                   
#     |______________v______________|      \-----/              
#

dx = 0.00001*500  # 10^-5 graus és aprox 1.11 m
dy = 0.00001*250

### Representa les geofences mòbils

In [ ]:
# def generaTrajectoriaGeofence(mapa,df):

In [ ]:
def representaTanquesMobil(mapa_servei,dfTraj_usuari):
    for i in range(len(dfTraj_usuari)):
        punt = (dfTraj_usuari.iloc[i][0],dfTraj_usuari.iloc[i][1])
        fence_mobil = geo.generaTancaMobil(punt,dx,dy)

        try: 
            folium.PolyLine(geo.Poligon_tancat(fence_mobil),
        # tooltip = f"""<div style = " font-size: 20px; font-family: courier new; color: blue">{mmsi} </div>""",
                            tooltip = "Date: %s"%dfTraj_usuari["DATETIME"].iloc[i],
                            popup="MMSI usuari: %s"%MMSI_usuari,  # El DF carregat no conté noms si està simplificat
                          # color = colors[tanques.index(tanca)],
                          # color = color_dict[-np.argwhere(MMSIs == mmsi)[0][0]%30],
                            color = "white",
        #                 # smooth_factor = (default: 1. més alt, més simplificat, baixa ressolució línies, millor reacció)
                            weight = 6, opacity=0.2,
                            smooth_factor = 10,
                           ).add_to(mapa_servei)
        except ValueErrror:print("error de valor")
    return(mapa_servei)
    
mapa_servei = geo.representaTanques(geo.regeneraMapa(cte.vlc0[0], zoom = 12),cte.tanques)    
representaTanquesMobil(mapa_servei,Trajectoria_usuari_df)
# # dfCargo['NAME'] = Fitxa["636017878"]['NAME'][0]#   'TIPOBUQUE'
# GuardaHTML(mapa,"Geofence_mobil_usuari")

In [ ]:
Trajectories=[Trajectoria_servei_df,Trajectoria_usuari_df]
colour = ["purple","orange"]

### Explicar-la
def cerclesData(mapa_servei,Trajectories,mmsi,a,b,colour = ["purple","orange"]):
    for Traj in range(len(Trajectories)):
        if Traj == 0:
            a = a
        else:
            a = b
        for i in range(0,len(Trajectories[Traj]),a):
            coord =(Trajectories[Traj]["LATITUD"].iloc[i],Trajectories[Traj]["LONGITUD"].iloc[i])
            folium.Circle(
                          location=(coord[0],coord[1]),
                          radius = 25,
                          color= colour[Traj], 
                          fill_color = colour[Traj],    
                          popup="Date: %s MMSI: %s"%(Trajectories[Traj]["DATETIME"].iloc[i],mmsi[Traj]), # falta imprimir mmsi
                        # El DF carregat no conté noms si està simplificat *
                #           tooltip = ,
                          smooth_factor = 40,  
                          ).add_to(mapa_servei)
    return(mapa_servei)
#  * dfCargo['NAME'] = Fitxa["636017878"]['NAME'][0], TIPOBUQUE'

# mapa_servei = geo.representaTanques(geo.regeneraMapa(cte.vlc0[0], zoom = 12),cte.tanques)    ## Regenerar
cerclesData(mapa_servei,Trajectories,[MMSI_usuari,MMSI_servei],1,1,colour)

# GuardaHTML(mapa,"Geofence_mobil_servei")

In [ ]:
geo.representaTrajectories(mapa_servei,[Trajectoria_usuari_df,Trajectoria_servei_df])
GuardaHTML(mapa_servei,"Geofence_mobil_complet")
mapa_servei

### Funció de cerca contacte usuari-servei

In [ ]:
### Per a cada geofence mòbil generada per l'usuari (totes les files de usuari)
### comprova si alguna coordenada del servei (totes les files de servei) està dins.*
### * Hi ha dos filtres de temps (+ SALT EN RANGE), primer deuriem haver eliminat les files per a les quals 
### sols hi ha una embarcació. Segon, sols comprova geofence i coordenades amb intervals de temps en comú (±30 min).
### He de limitar amb la data i en range perquè tarda molt. 1 minut cada 100 files del practic (poden haver > 6.000)

def ServeiPractic(Trajectoria_servei_df,Trajectoria_usuari_df,imprimir):
    i_servei_trobat, i_usuari_trobat = [],[]                            # Guardarà index fila amb coincidència
    #### *se salta passes per alleujerar!!!
    for i_servei in range(0,len(Trajectoria_servei_df),5):              # Recorre tots* els punts trajectoria servei
        for i_usuari in range(0,len(Trajectoria_usuari_df),5):          # Anàlogament per a usuari ######## *idem
            # Coordenades punt per fer geofence mòbil al voltant
            df_usuari_i = Trajectoria_usuari_df.iloc[i_usuari]
            df_servei_i = Trajectoria_servei_df.iloc[i_servei]
            punt = (df_usuari_i["LATITUD"],df_usuari_i["LONGITUD"])
            t_pos_usuari  = df_usuari_i["DATETIME"]         # Datetime del punt
            t_pos_servei  = df_servei_i["DATETIME"]                
            prev = t_pos_usuari - pd.to_timedelta("00:30:00", errors = "coerce")    # Temps ± 30 min filtrar posicions al mateix moment
            post = t_pos_usuari + pd.to_timedelta("00:30:00", errors = "coerce")
            if (prev < t_pos_servei < post):        # Sols comprovem si els temps coincideixen al menys mateixa hora
                fence_mobil = geo.generaTancaMobil(punt,dx,dy)                     # Geometria geofence mòbil
                trobat = len(geo.CoordenadaAtrapat([(fence_mobil,"fence_mobil")],  # retorna checklist de tanques
                                                     df_servei_i[["LATITUD","LONGITUD"]],
                                                     ver = False)) > 0   # trobat==True/False 
                if trobat:                         # Si ha entrat en geofence_mòbil imprimirà el Datetime del cargo
                    i_servei_trobat.append(i_servei)
                    i_usuari_trobat.append(i_usuari)
                    if imprimir: print(t_pos_usuari, "MMSI usuari", MMSI_usuari,
                          "Coordenades: (", punt[0],",",punt[1],")")   # Amb moltes coincidències PRINT AFEGIX TEMPS CÀLCUL
    if len(i_servei_trobat)>0:
        return(set(i_servei_trobat), set(i_usuari_trobat))

### Exemple cerca de contacte servei-usuari

In [ ]:
index_en_servei,index_en_usuari = ServeiPractic(Trajectoria_servei_df,Trajectoria_usuari_df, imprimir = True)
print(index_en_servei,index_en_usuari)

In [ ]:
mapa_servei = geo.representaTanques(geo.regeneraMapa(cte.vlc0[0], zoom = 12),cte.tanques)    
representaTanquesMobil(mapa_servei,Trajectoria_usuari_df.iloc[list(index_en_usuari)]) 
cerclesData(mapa_servei,[Trajectoria_servei_df.iloc[list(index_en_servei)]])

### Bucle per trobar contacte donat un usuari entre totes les embarcacions servei

In [ ]:
## Ací anirien bucles per trobar contactes amb servei practicatge. (tots mmsi tipus 70 per exemple i tots pilots)
##   Fer també bucles per trobar contactes amb servei de remolc.   (tots mmsi tipus 70 i tots remolcadors)

## vore al mapa totes traject tugs

In [ ]:
mapes, mmsi_tugs_no_error  = [],[]
for mmsi_servei in MMSIs_tugs: #: #MMSI_pilots:
    Traj_usu = geo.trajectoria(dfUDP_25, MMSI_usuari)
    Traj_ser = geo.trajectoria(dfUDP_25, mmsi_servei)
    Traj_ser, Traj_usu = retallarTemps(Traj_ser,Traj_usu)
    try:
        mapa = geo.representaTanques(geo.regeneraMapa(cte.vlc0[0], zoom = 12),cte.tanques)
        mapa = geo.representaTrajectories(mapa,[Traj_usu,Traj_ser])
        mapa = representaTanquesMobil(mapa,Traj_usu)
        mapa = cerclesData(mapa,[Traj_ser,Traj_usu],[MMSI_usuari,mmsi_servei])
        mapes.append(mapa)
        mmsi_tugs_no_error.append(mmsi_servei)
    except TypeError:
        print("\n Error en MMSI ",mmsi_servei,"\n")

In [ ]:
for mapa in mapes:
    GuardaHTML(mapa,"Geofence_mobil_servei_"+str(mapes.index(mapa)))

### recorre tugs no error
## Primer prova amb 1 per si apareix cap error

In [ ]:
# mmsi_servei=mmsi_tugs_no_error[2]
# Traj_usu = geo.trajectoria(dfUDP_25, MMSI_usuari)
# Traj_ser = geo.trajectoria(dfUDP_25, mmsi_servei)
# Traj_ser, Traj_usu = retallarTemps(Traj_ser,Traj_usu)
# try:
#     index_en_servei,index_en_usuari = ServeiPractic(Traj_ser,Traj_usu, imprimir = True)
# except TypeError:
#     print("\nNo hi ha cap coincidència\n")

In [ ]:
print(DataInstant())
for mmsi_servei in mmsi_tugs_no_error:
    print("\n",MMSI_usuari,mmsi_servei)
    Traj_usu = geo.trajectoria(dfUDP_25, MMSI_usuari)
    Traj_ser = geo.trajectoria(dfUDP_25, mmsi_servei)
    Traj_ser, Traj_usu = retallarTemps(Traj_ser,Traj_usu)
    try:
        index_en_servei,index_en_usuari = ServeiPractic(Traj_ser,Traj_usu, imprimir = True)
        print(index_en_servei,index_en_usuari)
        mapa_servei = geo.representaTanques(geo.regeneraMapa(cte.vlc0[0], zoom = 12),cte.tanques)    
        representaTanquesMobil(mapa_servei,Traj_usu.iloc[list(index_en_usuari)]) 
        cerclesData(mapa_servei,[Traj_ser.iloc[list(index_en_servei)],Traj_usu.iloc[list(index_en_usuari)]],1,1,[MMSI_usuari,mmsi_servei],colour)
        GuardaHTML(mapa_servei,"Geofence_mobil_servei-usuari_"+str(mmsi_tugs_no_error.index(mmsi_servei)))
    except TypeError:
        print("\nNo hi ha cap coincidència\n")
print(DataInstant())    

In [ ]:
#### Exemple de marcadors i eina per seleccionar-los

# tooltip = "Click me!"

# folium.Marker(
#     [+39.4316,-0.3316],
#     popup="<i>Latitut mínima en requests API</i>", tooltip=tooltip
# ).add_to(m)

# folium.Marker(
#     location=[+39.44,-0.3316],
#     popup="Mt. Hood Meadows",
#     icon=folium.Icon(icon="cloud",color="orange"),
# ).add_to(m)


# m